# 2.3 Limpeza de Dados

### Terceira Limpeza

In [1]:
# Biblioteca
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',60)

## Dados Constantes

In [31]:
DATA_OUTPUT_RAW = '../data/DATA_baixada_santista_raw.csv'
DATA_OUTPUT_TRAIN_CLEANED = '../data/DATA_train_cleaned.csv'
DATA_TEST = '../data/DATA_TEST.csv'
TARGET = 'Preço/Noite'

In [3]:
df_raw = pd.read_csv(DATA_OUTPUT_RAW)

In [4]:
df_raw.head()

,Unnamed: 0,ID,Check-In,Check-Out,Localização,titulo_2,Número Hóspedes,Número Quartos,Número Camas,Número Banheiros,Plus 1,Plus 2,Plus 3,Plus 4,Avaliação,Número Comentários,Superhost,Preço/Noite,Preço com taxas
0,0,20193688,2021-02-26,2021-02-28,Jardim Las Palmas,Astúrias Apto aconchegante perto da praia,2,1,1,1,Ar-condicionado,Piscina,Cozinha,Estacionamento gratuito,4.82,46,Novo preço mais baixo,228.0,456.0
1,1,37237280,2021-02-26,2021-02-28,Pitangueiras,Suíte Guarujá apto Frente Mar 2,7,1,4,1 e meio,Wi-Fi,Cozinha,Permitido animais,Elevador,4.67,21,Novo preço mais baixo,185.0,369.0
2,2,44419309,2021-02-26,2021-02-28,José Menino,Charmoso apartamento em Santos na quadra da praia,4,1,1,1,Wi-Fi,Ar-condicionado,Cozinha,Estacionamento gratuito,4.84,25,Novo preço mais baixo,171.0,342.0
3,3,39763299,2021-02-26,2021-02-28,José Menino,"Conforto,frente à praia do José menino/Santos",3,Estúdio,2,1,Cozinha,Permitido animais,Elevador,Vista para o mar,4.65,51,Novo preço mais baixo,108.0,296.0
4,4,20774709,2021-02-26,2021-02-28,"Guarujá, Pitangueiras Centro",Guarujá Centro Pitangueiras Wi-Fi,4,1,3,1,Cozinha,Estacionamento gratuito,Permitido animais,Elevador,4.49,97,Novo preço mais baixo,188.0,456.0


In [5]:
df_raw.shape

(5262, 19)

### Linhas duplicadas

In [6]:
print(f' Linhas duplicadas: {df_raw.duplicated().sum()}')

 Linhas duplicadas: 0


### Colunas duplicadas

In [7]:
df_raw_T = df_raw.T
df_raw_T[df_raw_T.duplicated(keep=False)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,...,5232,5233,5234,5235,5236,5237,5238,5239,5240,5241,5242,5243,5244,5245,5246,5247,5248,5249,5250,5251,5252,5253,5254,5255,5256,5257,5258,5259,5260,5261


### Colunas Constantes

In [8]:
columns_to_drop_constant = []
for col in df_raw.columns:
    if df_raw[col].nunique() == 1:
        columns_to_drop_constant.append(col)
    else:
        continue

In [9]:
columns_to_drop_constant

[]

## 2.3.1. Divisão dos Dados

In [11]:
df_train, df_test = train_test_split(df_raw,
                                     test_size=0.2,
                                     random_state=123)
df_train.shape, df_test.shape

((4209, 19), (1053, 19))

## 2.3.2 Damificação de variáveis

### Criação de Dummies

Ainda temos que criar uma função que coloque no For todos os novas palavras que possam vir com atualizações do dataset. Por hora essa lista está estática, o ideal é que ela seja dinâmica

In [14]:
def create_dummies(df):
    """
    Create dummies from Plus and Superhost columns
    df = DataFrame from airbnb scrapping
    Return DataFrame with dummies columns and dropped the originals
    """ 
    # Dummies das colunas Plus
    plus1 = df['Plus 1'].unique()
    plus2 = df['Plus 2'].unique()
    plus3 = df['Plus 3'].unique()
    plus4 = df['Plus 3'].unique()
    fullplus = plus1.tolist() + plus2.tolist() + plus3.tolist() + plus4.tolist()
    fullplus = set(fullplus)
    def word_in_columns(df, word):
        if word in df['Plus 1'] or word in df['Plus 2'] or word in df['Plus 3'] or word in df['Plus 4']:
            return 1
        else:
            return 0
        
    count = 0
    for word in fullplus:
        df[f'{word}'] = df.apply(word_in_columns, axis=1, args= (word, ))
        count += 1
    
    # Dummies para Superhost
    Superhost = df['Superhost'].unique()
    def host_in_columns(df, word):
        if word in df['Superhost']:
            return 1
        else:
            return 0
        
    count = 0
    for word in Superhost:
        df[f'{word}'] = df.apply(word_in_columns, axis=1, args= (word, ))
        count += 1
    
    #Drop das colunas 
    df = df.drop(columns=['Plus 1' , 'Plus 2', 'Plus 3', 'Plus 4', 'Superhost'])
    
    return df

In [15]:
df_train = create_dummies(df_train)

<ipython-input-14-7b2874214b85>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{word}'] = df.apply(word_in_columns, axis=1, args= (word, ))
<ipython-input-14-7b2874214b85>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{word}'] = df.apply(word_in_columns, axis=1, args= (word, ))


In [16]:
df_train.head()

,Unnamed: 0,ID,Check-In,Check-Out,Localização,titulo_2,Número Hóspedes,Número Quartos,Número Camas,Número Banheiros,Avaliação,Número Comentários,Preço/Noite,Preço com taxas,Vista para o mar,Piscina,Lava-louças,Wi-Fi,Café da Manhã,Máquina de Lavar,Estacionamento gratuito,Academia,Jacuzzi,Secadora,Entrada/saída para esquis,Cozinha,Vista para as águas,Permitido animais,Lareira interna,Ar-condicionado,Self check-In,Elevador,Novo preço mais baixo,Raridade
550,550,39499342,2021-03-05,2021-03-07,Praia Grande,Apartamento 1 dormitório no Boqueirão com Pisc...,4,1,3,1,5.00,18,184.0,488.0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
682,682,40945881,2021-03-19,2021-03-21,Guarujá,"Guarujá, praia 1 quadra, Wi-Fi, ar Split - 1002",4,1,4,1,4.83,41,198.0,456.0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
5224,7621,46875225,2021-03-05,2021-03-07,Casa inteira em Parque Balneario Oasis,Casa Rosa de Peruíbe,10,3,8,2,4.43,7,351.0,702.0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1172,1188,46743206,2021-03-26,2021-03-28,Vila Tupi,Lindo apartamento na praia da Praia Grande,4,1,2,1,4.40,5,191.0,502.0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0
3857,5648,46866361,2021-02-26,2021-02-28,Vila Tupi,#Kitnet63 | A felicidade está a 150 m da praia! ❤,4,1,4,1,5.00,3,124.0,364.0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0


### Tratamento do Número de Quartos

In [17]:
def quartos_transformation(df):
    df.replace({'Número Quartos' : {'Estúdio': '1'}}, inplace=True)
    return df

In [18]:
df_train = quartos_transformation(df_train)

### Tratamento do Número de banheiros

In [19]:
# verificando as inconsistências:

df_train['Número Banheiros'].value_counts()

1                         2674
2                          764
3                          191
1 e meio                   189
1 privado                  171
2 e meio                    93
4                           29
3 e meio                    28
1 compartilhado             18
5 e meio                    16
5                            9
2 compartilhados             7
4 e meio                     6
0                            4
6 e meio                     4
6                            4
1 compartilhado e meio       2
Name: Número Banheiros, dtype: int64

**Função de tratamento de problemas**

In [20]:
def banheiro_transformation(df):
    """
    Extract any word inside Número de banheiros column and create a new one called Banheiro Compartilhado.
    In banheiro Compartilhado: If True, There is Banheiro compartilhado. If False, There is no Banheiro compartilhado.
    df = raw dataset cleanned from airbnb scrapping
    """
    # Pega o qualquer texto após o espaço.
    df['Banheiro Compartilhado'] = df['Número Banheiros'].str.contains('\\ ', regex=True)
    
    # Pega o primeiro digito numérico
    df['Número Banheiros'] = df['Número Banheiros'].str[0:1]
    df['Número Banheiros'] = df['Número Banheiros'].str.strip()
    
    return df

In [21]:
df_train = banheiro_transformation(df_train)

In [22]:
df_train['Número Banheiros'].value_counts()

1    3054
2     864
3     219
4      35
5      25
6       8
0       4
Name: Número Banheiros, dtype: int64

## 2.3.2 Tratamento de variáveis

In [23]:
# Vamos dropar a coluna Unnamed: 0 e  Titulo_2
df_train = df_train.drop({'Unnamed: 0', 'titulo_2'}, axis= 1)

In [24]:
# Análise de Variáveis
df_train.dtypes

ID                             int64
Check-In                      object
Check-Out                     object
Localização                   object
Número Hóspedes                int64
Número Quartos                object
Número Camas                   int64
Número Banheiros              object
Avaliação                    float64
Número Comentários             int64
Preço/Noite                  float64
Preço com taxas              float64
Vista para o mar               int64
Piscina                        int64
Lava-louças                    int64
Wi-Fi                          int64
Café da Manhã                  int64
Máquina de Lavar               int64
Estacionamento gratuito        int64
Academia                       int64
Jacuzzi                        int64
Secadora                       int64
Entrada/saída para esquis      int64
Cozinha                        int64
Vista para as águas            int64
Permitido animais              int64
Lareira interna                int64
A

### Ajuste dos Tipos

In [22]:
# A variável número de quartos e número de banheiros estavam como categóricas. Então, fez se necessário a conversão para numérica

# Transformação em datatime as variáveis Check-in e Check-out:

In [25]:
def ajuste_tipos(df):
    """
    Types transformation
    df= dataset
    """
    
    # Object to Numeric
    df['Número Quartos'] = pd.to_numeric(df['Número Quartos'], errors = 'coerce')
    df['Número Banheiros'] = pd.to_numeric(df['Número Banheiros'], errors = 'coerce')
    # To DateTime
    df['Check-In'] = pd.to_datetime(df_train['Check-In'], format ='%Y-%m-%d')
    df['Check-Out'] = pd.to_datetime(df_train['Check-Out'], format ='%Y-%m-%d')
    
    return df

In [26]:
df_train = ajuste_tipos(df_train)

In [27]:
# Agora todos os tipos estão classificados corretamente.
df_train.dtypes

ID                                    int64
Check-In                     datetime64[ns]
Check-Out                    datetime64[ns]
Localização                          object
Número Hóspedes                       int64
Número Quartos                        int64
Número Camas                          int64
Número Banheiros                      int64
Avaliação                           float64
Número Comentários                    int64
Preço/Noite                         float64
Preço com taxas                     float64
Vista para o mar                      int64
Piscina                               int64
Lava-louças                           int64
Wi-Fi                                 int64
Café da Manhã                         int64
Máquina de Lavar                      int64
Estacionamento gratuito               int64
Academia                              int64
Jacuzzi                               int64
Secadora                              int64
Entrada/saída para esquis       

In [28]:
# Checando valores atípicos
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,4209.0,3.607509e+07,1.056366e+07,848531.0,30400702.00,39646858.00,45112809.00,47834074.0
Número Hóspedes,4209.0,5.034688e+00,2.413963e+00,2.0,4.00,4.00,6.00,16.0
Número Quartos,4209.0,1.433832e+00,7.463190e-01,1.0,1.00,1.00,2.00,5.0
Número Camas,4209.0,3.165598e+00,2.251931e+00,0.0,2.00,3.00,4.00,16.0
Número Banheiros,4209.0,1.366595e+00,7.053138e-01,0.0,1.00,1.00,2.00,6.0
Avaliação,4209.0,4.829499e+00,1.895130e-01,3.0,4.75,4.89,4.97,5.0
Número Comentários,4209.0,3.652602e+01,3.718458e+01,3.0,11.00,26.00,47.00,281.0
Preço/Noite,4209.0,2.545158e+02,1.475012e+02,69.0,162.00,213.00,294.00,1495.0
Preço com taxas,4209.0,5.923939e+02,3.238689e+02,114.0,388.00,514.00,685.00,3344.0
Vista para o mar,4209.0,1.857923e-01,3.889852e-01,0.0,0.00,0.00,0.00,1.0


Valores atítpicos identificados: <br>
Há quartos em que não 0 camas e 0 Banheiros.

In [29]:
# Invetigando o número de camas.
df_train['Número Camas'].value_counts()

2     1195
3      833
1      825
4      581
5      214
6      192
7      126
8       68
9       65
10      34
0       22
11      19
16      15
12      12
15       8
Name: Número Camas, dtype: int64

In [30]:
# Supondo que se há quarto é possivel que haja cama.
len(df_train.loc[(df_train['Número Camas'] == 0 & (df_train['Número Quartos'] >=1))])

22

Entretanto, Existe a possibilidade do locatário alugar o imóvel por temporada sem disponibilizar uma cama.<br>
Pesando nesta linha, vamos optar por não alterar e/ou remover estes imóveis

# Exportar

In [32]:
df_train.to_csv(DATA_OUTPUT_TRAIN_CLEANED)
df_test.to_csv(DATA_TEST)